In [ ]:
def limb_tour(Camera_ID, whatever,bufArray, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade) -> None:
    #print("L138, Camera_ID, whatever, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade")
    #print(Camera_ID, whatever, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
    #print("L124 gain ",gain)#scan arround the circle Scope starts centered on sun as seen in chart center and return there
    #gain=150
    limb=np.zeros((5000,5000))#used to store limb tour images ((imgHeight,imgWidth))
    f = whatever + "limb.png"               
    cv2.imwrite(f,limb)
    
    #https://www.codeease.net/programming/python/draw-circle-with-numpy
    # Create a numpy array of angles from 0 to 2*pi
    angles = np.linspace(0, 2*np.pi, int(24/imagescale))
    r=0.009*2+0.0025    #0.0045*4[0.009*60*60 = 32 Arc sec sun diameter]why 2x off?
    #sensor 6.97x5.32[arcmin] FOV is imgWidth0ximgHeight [pixels]
    #ODsun 32 [ArcMin] FOV
    #camer width/sunwidth=6.97/32=imgWidth/sunpixels 
    SunPixelsWidth=imgWidth*32/6.97 # = 2,938 [9 um pixels]
    crp=1673        #(0.018+0.0025)/0.018*(2,938/2) [radius of sun in 9 um pixeles]
    # Create a numpy array of x-coordinates of the circle
    rscale=2.5
    x = (r*np.cos(angles)/rscale)
    cxp = (crp*np.cos(angles)/rscale)
    # Create a numpy array of y-coordinates of the circle
    y = (r*np.sin(angles)/rscale)
    cyp = (crp*np.sin(angles)/rscale)
    i=0
    tm=3
    for ang in angles: 
        data = cdccmd(b'GETRA F').decode('utf-8')
        CRTRAF=0.1
        CRTRAF=extract_floats(data)
        print ("L119 getchartra = cdccmd(b'GETRA F ') ",CRTRAF )
        data= cdccmd(b'GETDEC F').decode('utf-8')
        CRTDEF=0.1
        CRTDEF=extract_floats(data)
        print ("L123 getchartde = cdccmd(b'GETDEC F ') ",CRTDEF )
                
        CRTrade = str(CRTRAF[0]) + " " + str(CRTDEF[0])
   
        c="MOVESCOPE "+ CRTrade#Yes MOVESCOPE name is misleading, it only show a mark on the chart.
        print("L129 " , c);    cdccmd(str.encode(c))
        rade = f"{CRTRAF[0]+float(x[i]):10f}" + " " + f"{CRTDEF[0]+float(y[i]*14.25):10f}"
        c="MOVESCOPE "+ rade;        print(c);        cdccmd(str.encode(c))
        TELrade = rade
        cdccmd(b'REDRAW')
        time.sleep(tm)
        
        c="SLEW "+ rade;        print(c);        cdccmd(str.encode(c));        time.sleep(tm)
        cdccmd(b'REDRAW');        time.sleep(tm)
        
        data=cdccmd(b'GETSCOPERADEC');        time.sleep(tm)
        print ("L177 cdccmd(b'GETSCOPERADEC')",data);   
        time.sleep(tm);     
        data = data.decode('utf-8');       
        print("L180 data.decode('utf-8') ",data)
        
        numbers = extract_signed_numbers(data)#  [3.0, 23.0, 24.0, -21.0, 17.0, 30.0]
        print ("L183 numbers ",numbers)
        
        raH=int(numbers[0]);            raM=int(numbers[1]);            raS=int(numbers[2]);        print("L162 raH ",raH," raM ",raM," raS ",raS)
        raF= f"{float(raH)+float(raM)/60+float(raS)/3600:.5f}"  
        deD=int(numbers[3]);            deM=int(numbers[4]);            deS=int(numbers[5]);        print("L150 deD ",deD,"deM ",deM, " deS ",deS)
        deF= f"{float(deD)+float(deM)/60+float(deS)/3600:.5f}";        print("L151 deD ", deD, "deM ", deM, " deS ", deS, " deF ", deF)
        print("GETSCOPERADEC " + str(raF) + " " + str(deF))
        TEOrade = str(raF) + " " + str(deF)
        print("L191")
        #pixelsperarcsec = 91.82209469153515 # RA imgWidth[pixels]/6.97[ArcMin] [pixels/ArcMin] Dec imgHeight/5.23, ODminsun 31.45 [ArcMinFOV] 
             
        time.sleep(tm)
        j=0
        SUNRISE_controls=np.zeros((480,640))
        greatestimageTrue=0
        time.sleep(tm)
        while(True):
            # if breakTrigger:
            #   break 
            Camera_Error,pIsReady = pyPOACamera.ImageReady(Camera_ID) # In Video Mode, only need to check if the image is ready, NO need to check camera state
            if pIsReady:
                break
        Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
        gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
        print("Camera_ID, whatever, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade")
        print(Camera_ID, whatever, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
        key=cv2.waitKey(1)#imshow() only works with waitKey()
        cv2.imshow("gray",gray)
        Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
        gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
        key=cv2.waitKey(1)#imshow() only works with waitKey()
        cv2.imshow("gray",gray)
        Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
        gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
        print(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade) 
        key=cv2.waitKey(1)#imshow() only works with waitKey()
        cv2.imshow("gray",gray)
        si=0
        j=0
        greatestimageTrue=0
        while j<60:
            # if breakTrigger:
            #   break
            while(True):
                # if breakTrigger:
                #   break 
                Camera_Error,pIsReady = pyPOACamera.ImageReady(Camera_ID) # In Video Mode, only need to check if the image is ready, NO need to check camera state
                if pIsReady:
                    break
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade) 
            cv2.imshow("gray",gray)
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
            cv2.imshow("gray",gray)
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
            cv2.imshow("gray",gray)
            #greatestimage=gray
            si=pixelssum(gray)
            if  si > greatestimageTrue:
                greatestimageTrue=si
                greatestimage=gray
                cv2.imshow("gray",greatestimage);    key=cv2.waitKey(1)#imshow() only works with waitKey()
            
            if key==ord('g'):
                gain=gain-1
                if gain <= 0:
                    gain = 0
                #print ("gain ",gain)
                Camera_Error = pyPOACamera.SetGain(Camera_ID,gain,False)      
            if key==ord('G'):
                gain=gain+1
                if gain <= 0:
                    gain = 0
                #print ("gain ",gain)
                Camera_Error = pyPOACamera.SetGain(Camera_ID,gain,False) 
            if key==ord('E'):
                exposure=exposure+100#17000
                if exposure >= 50000:
                    exposure = 50000
                pyPOACamera.SetExp(Camera_ID,  exposure, False) # set exposure to 17ms
            if key==ord('e'):
                exposure=exposure-100
                if exposure <= 100:
                    exposure = 100
                pyPOACamera.SetExp(Camera_ID, exposure, False) # set exposure to 17ms   
            if key==ord('t'):
                threshold=threshold-100
                if threshold <= 0:
                    threshold = 0
            if key==ord('T'):
                threshold=threshold+1
                if threshold <= 0:
                    threshold = 0
            if key==ord('f'):
                fav=fav-0.1
                if fav <= 0:
                    fav = 0
            if key==ord('F'):
                fav=fav+0.1
                if fav <= 0:
                    fav = 0
            if key==ord('i'):
                offsetsumedge=offsetsumedge-100
                if offsetsumedge <= 0:
                    offsetsumedge = 1
            if key==ord('I'):
                offsetsumedge=offsetsumedge+100
                if offsetsumedge <= 0:
                    offsetsumedge = 1
            if key==ord('x'):
                Contrast_control_X=Contrast_control_X-0.1
                if Contrast_control_X <= 0:
                    Contrast_control_X = 0
            if key==ord('X'):
                Contrast_control_X=Contrast_control_X+0.1
                if Contrast_control_X <= 0:
                    Contrast_control_X = 0
            if key==ord('v'):
                Brightness_control_V=Brightness_control_V-0.1
                if Brightness_control_V <= 0:
                    Brightness_control_V= 0
            if key==ord('V'):
                Brightness_control_V=Brightness_control_V+0.1
                if Brightness_control_V <= 0:
                    Brightness_control_V = 0
            if key==ord('l'):
                Lthreshold=Lthreshold-1                #print ("Lthreshold ",Lthreshold)
            if Lthreshold <= 0:
                Lthreshold = 0
            if key==ord('L'):
                Lthreshold=Lthreshold+1
                if Lthreshold <= 0:
                    Lthreshold = 0
                #print ("Lthreshold ",Lthreshold)
            if key==ord('u'):
                u5=u5-1
                if u5 <= 0:
                    u5 = 0                #print ("u5 ",u5)
            if key==ord('U'):
                u5=u5+1
                if u5 <= 0:
                    u5 = 0                #print ("u5 ",u5)
            #img_scaled_greatestimage = cv2.normalize(sharpestimage, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)
            cv2.imshow("gray",greatestimage)
            im_color = cv2.applyColorMap(greatestimage, cv2.COLORMAP_HOT)
            cv2.imshow('HOT ',im_color) 
            cv2.putText(img=SUNRISE_controls, text="TELrade[Hr Deg]: "+str(TELrade),org=(0, 255), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="TELrade[Hr Deg]: "+str(TELrade),org=(0, 255), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)
            cv2.putText(img=SUNRISE_controls, text="TEOrade[Hr Deg]: "+str(TEOrade),org=(0, 270), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="TEOrade[Hr Deg]: "+str(TEOrade),org=(0, 270), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)
            cv2.putText(img=SUNRISE_controls, text="CRTrade[Hr Deg]: "+str(CRTrade),org=(0, 285), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="CRTrade[Hr Deg]: "+str(CRTrade),org=(0, 285), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)      
            
            cv2.putText(img=SUNRISE_controls, text="cxp,cyp: " + str(int(cxp[i]))+" " + str(int(cyp[i])),org=(0, 315), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="cxp,cyp: " + str(int(cxp[i]))+" " + str(int(cyp[i])),org=(0, 315), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)      

            cv2.imshow("SUNRISE_controls: SUN surface and flare Real-time Imaging systEm",SUNRISE_controls)
            cv2.imshow("SUNRISE_histogram: ",SUNRISE_histogram)
            #print("L258")
            j=j+1    
        back = cv2.imread( whatever + "limb.png")
        pixelcenterlocation=" " + str(int(cxp[i])) + " " + str(int(cyp[i]))
        f = save_gray_image(greatestimage,SUNRISE_controls,pixelcenterlocation)
        overlay = cv2.imread(f)
        h, w = back.shape[:2]
        print(h, w)
        h1, w1 = overlay.shape[:2]
        print(h1, w1)

        # let store center coordinate as cx,cy
        cx, cy = (h - h1) // 2 - int(cxp[i]), (w - w1) // 2 - int(cyp[i])
        print (cx)
        print (cy)
        # use numpy indexing to place the resized image in the center of 
        # background image
        back[cy:cy + h1, cx:cx + w1] = overlay
        # save resulting centered image
        cv2.imwrite(whatever + "limb.png", back)
        time.sleep(2)
        i=i+1
        
    #CRTrade = str(CRTRAF[0]) + " " + str(CRTDEF[0])
    c="MOVESCOPE "+ CRTrade#Yes MOVESCOPE name is misleading, it only show a mark on the chart.
    print(c);    cdccmd(str.encode(c))
    cdccmd(b'REDRAW');    time.sleep(tm)
    c="SLEW "+ CRTrade;    print(c);    cdccmd(str.encode(c))
    time.sleep(tm)
    cdccmd(b'REDRAW')
    time.sleep(tm)

    #https://www.codeease.net/programming/python/draw-circle-with-numpy
    # Create a numpy array of angles from 0 to 2*pi
    angles = np.linspace(0, 2*np.pi, int(24/imagescale))
    r=0.009*2+0.0025    #0.0045*4[0.009*60*60 = 32 Arc sec sun diameter]why 2x off?
    #sensor 6.97x5.32[arcmin] FOV is imgWidth0ximgHeight [pixels]
    #ODsun 32 [ArcMin] FOV
    #camer width/sunwidth=6.97/32=imgWidth/sunpixels 
    SunPixelsWidth=imgWidth*32/6.97 # = 2,938 [9 um pixels]
    crp=int(1673*0.95)       #(0.018+0.0025)/0.018*(2,938/2) [radius of sun in 9 um pixeles]
    # Create a numpy array of x-coordinates of the circle
    rscale=1
    gain=173
    x = r*np.cos(angles)/rscale
    cxp = crp*np.cos(angles)/rscale
    # Create a numpy array of y-coordinates of the circle
    y = r*np.sin(angles)/rscale
    cyp = crp*np.sin(angles)/rscale
    i=0
    for ang in angles: 
        data = cdccmd(b'GETRA F').decode('utf-8')
        time.sleep(tm)
        CRTRAF=0.1
        CRTRAF=extract_floats(data)
        print ("L104 getchartra = cdccmd(b'GETRA F ') ",CRTRAF )
        data= cdccmd(b'GETDEC F').decode('utf-8')
        time.sleep(tm)
        CRTDEF=0.1
        CRTDEF=extract_floats(data)
        
        print ("L108 getchartde = cdccmd(b'GETDEC F ') ",CRTDEF )        
        CRTrade = str(CRTRAF[0]) + " " + str(CRTDEF[0])
        c="MOVESCOPE "+ CRTrade#Yes MOVESCOPE name is misleading, it only show a mark on the chart.
        print("L113 " , c);    cdccmd(str.encode(c))
        rade = f"{CRTRAF[0]+float(x[i]):10f}" + " " + f"{CRTDEF[0]+float(y[i]*14.25):10f}"
        c="MOVESCOPE "+ rade;        print(c);        cdccmd(str.encode(c))
        TELrade = rade
        cdccmd(b'REDRAW')
        time.sleep(tm)
        
        c="SLEW "+ rade;        print(c);        cdccmd(str.encode(c));        time.sleep(tm)
        cdccmd(b'REDRAW');        time.sleep(tm)
        
        data=cdccmd(b'GETSCOPERADEC');        time.sleep(tm)
        print ("L138 cdccmd(b'GETSCOPERADEC')",data);   
        time.sleep(tm);     
        data = data.decode('utf-8');        
        print("L416 data.decode('utf-8') ",data)
        
        numbers = extract_signed_numbers(data)#  [3.0, 23.0, 24.0, -21.0, 17.0, 30.0]
        print ("L419 numbers ",numbers)
        
        raH=int(numbers[0]);            raM=int(numbers[tm]);            raS=int(numbers[2]);        print("L344 raH ",raH," raM ",raM," raS ",raS)
        raF= f"{float(raH)+float(raM)/60+float(raS)/3600:.5f}"  
        deD=int(numbers[3]);            deM=int(numbers[4]);            deS=int(numbers[5]);        print("L347 deD ",deD,"deM ",deM, " deS ",deS)
        data=cdccmd(b'GETSCOPERADEC')
        time.sleep(tm)
        print ("L426 cdccmd(b'GETSCOPERADEC')",data)  
        
        print("L428 raH ",raH," raM ",raM," raS ",raS)
        raF= f"{float(raH)+float(raM)/60+float(raS)/3600:.5f}" ;    print("L358 deD ",deD,"deM ",deM, " deS ",deS)

        deF= f"{float(deD)+float(deM)/60+float(deS)/3600:.5f}" 
        print("L432 deD ", deD, "deM ", deM, " deS ", deS, " deF ", deF)
        print("GETSCOPERADEC " + str(raF) + " " + str(deF))
        TEOrade = str(raF) + " " + str(deF)
        print("L435")
        #pixelsperarcsec = 91.82209469153515 # RA imgWidth[pixels]/6.97[ArcMin] [pixels/ArcMin] Dec imgHeight/5.23, ODminsun 31.45 [ArcMinFOV] 
             
        j=0
        si=0
        SUNRISE_controls=np.zeros((480,640))
        greatestimageTrue=0
        while j<60:
            # if breakTrigger:
            #   break
            while(True):
                # if breakTrigger:
                #   break 
                Camera_Error,pIsReady = pyPOACamera.ImageReady(Camera_ID) # In Video Mode, only need to check if the image is ready, NO need to check camera state
                if pIsReady:
                    break
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade) 
            cv2.imshow("gray",gray)
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
            cv2.imshow("gray",gray)
            Camera_Error = pyPOACamera.GetImageData(Camera_ID, bufArray, 1000) # GetImageData needs to prepare the bufArray first
            gray=get_gray_image(Camera_ID, whatever,bufArray, cl, TGS, gain, exposure, threshold, fav, offsetsumedge, Contrast_control_X,Brightness_control_V,Lthreshold,u5,SUNRISE_controls,CRTrade)
            cv2.imshow("gray",gray)

            si=pixelssum(gray)
            if  si > greatestimageTrue:
                greatestimageTrue=si
                greatestimage=gray
                cv2.imshow("gray",greatestimage)

            key=cv2.waitKey(1)#imshow() only works with waitKey()
            
            if  si > greatestimageTrue:
                greatestimageimageTrue=si
                greatestimage=gray
            if key==ord('g'):
                gain=gain-1
                if gain <= 0:
                    gain = 0
                print ("gain ",gain)
                Camera_Error = pyPOACamera.SetGain(Camera_ID,gain,False)      
            if key==ord('G'):
                gain=gain+1
                if gain <= 0:
                    gain = 0
                print ("gain ",gain)
                Camera_Error = pyPOACamera.SetGain(Camera_ID,gain,False) 
            if key==ord('E'):
                exposure=exposure+100#17000
                if exposure >= 50000:
                    exposure = 50000
                    pyPOACamera.SetExp(Camera_ID,  exposure, False) # set exposure to 17ms
            if key==ord('e'):
                exposure=exposure-100
                if exposure <= 100:
                    exposure = 100
                pyPOACamera.SetExp(Camera_ID, exposure, False) # set exposure to 17ms   
            if key==ord('t'):
                threshold=threshold-100
                if threshold <= 0:
                    threshold = 0
            if key==ord('T'):
                threshold=threshold+1
                if threshold <= 0:
                    threshold = 0
            if key==ord('f'):
                fav=fav-0.1
                if fav <= 0:
                    fav = 0
            if key==ord('F'):
                fav=fav+0.1
                if fav <= 0:
                    fav = 0
            if key==ord('i'):
                offsetsumedge=offsetsumedge-100
                if offsetsumedge <= 0:
                    offsetsumedge = 1
            if key==ord('I'):
                offsetsumedge=offsetsumedge+100
                if offsetsumedge <= 0:
                    offsetsumedge = 1
            if key==ord('x'):
                Contrast_control_X=Contrast_control_X-0.1
                if Contrast_control_X <= 0:
                    Contrast_control_X = 0
            if key==ord('X'):
                Contrast_control_X=Contrast_control_X+0.1
                if Contrast_control_X <= 0:
                    Contrast_control_X = 0
            if key==ord('v'):
                Brightness_control_V=Brightness_control_V-0.1
                if Brightness_control_V <= 0:
                    Brightness_control_V= 0
            if key==ord('V'):
                Brightness_control_V=Brightness_control_V+0.1
                if Brightness_control_V <= 0:
                    Brightness_control_V = 0
            if key==ord('l'):
                Lthreshold=Lthreshold-1                #print ("Lthreshold ",Lthreshold)
            if Lthreshold <= 0:
                Lthreshold = 0
            if key==ord('L'):
                Lthreshold=Lthreshold+1
                if Lthreshold <= 0:
                    Lthreshold = 0
                #print ("Lthreshold ",Lthreshold)
            if key==ord('u'):
                u5=u5-1
                if u5 <= 0:
                    u5 = 0                #print ("u5 ",u5)
            if key==ord('U'):
                u5=u5+1
                if u5 <= 0:
                    u5 = 0                #print ("u5 ",u5)
            #img_scaled_sharpestimage = cv2.normalize(sharpestimage, dst=None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)
            cv2.imshow("gray",greatestimage)
            im_color = cv2.applyColorMap(greatestimage, cv2.COLORMAP_HOT)
            cv2.imshow('HOT ',im_color) 
            cv2.putText(img=SUNRISE_controls, text="TELrade[Hr Deg]: "+str(TELrade),org=(0, 255), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="TELrade[Hr Deg]: "+str(TELrade),org=(0, 255), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)
            cv2.putText(img=SUNRISE_controls, text="TEOrade[Hr Deg]: "+str(TEOrade),org=(0, 270), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="TEOrade[Hr Deg]: "+str(TEOrade),org=(0, 270), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)
            cv2.putText(img=SUNRISE_controls, text="CRTrade[Hr Deg]: "+str(CRTrade),org=(0, 285), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="CRTrade[Hr Deg]: "+str(CRTrade),org=(0, 285), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)      
            
            cv2.putText(img=SUNRISE_controls, text="cxp,cyp: " + str(int(cxp[i]))+" " + str(int(cyp[i])),org=(0, 315), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 0),thickness=2)
            cv2.putText(img=SUNRISE_controls, text="cxp,cyp: " + str(int(cxp[i]))+" " + str(int(cyp[i])),org=(0, 315), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(250, 250, 250),thickness=1)      

            cv2.imshow("SUNRISE_controls: SUN surface and flare Real-time Imaging systEm",SUNRISE_controls)
            cv2.imshow("SUNRISE_histogram: ",SUNRISE_histogram)
            #print("L258")
            j=j+1    
        back = cv2.imread( whatever + "limb.png")
        pixelcenterlocation=" " + str(int(cxp[i])) + " " + str(int(cyp[i]))
        f = save_gray_image(greatestimage,SUNRISE_controls,pixelcenterlocation)
        overlay = cv2.imread(f)
        h, w = back.shape[:2]
        print(h, w)
        h1, w1 = overlay.shape[:2]
        print(h1, w1)

        # let store center coordinate as cx,cy
        cx, cy = (h - h1) // 2 - int(cxp[i]), (w - w1) // 2 - int(cyp[i])
        print (cx)
        print (cy)
        # use numpy indexing to place the resized image in the center of 
        # background image
        back[cy:cy + h1, cx:cx + w1] = overlay
        # save resulting centered image
        cv2.imwrite(whatever + "limb.png", back)
        time.sleep(5)
        i=i+1
        
    #CRTrade = str(CRTRAF[0]) + " " + str(CRTDEF[0])
    c="MOVESCOPE "+ CRTrade#Yes MOVESCOPE name is misleading, it only show a mark on the chart.
    print(c);    cdccmd(str.encode(c))
    cdccmd(b'REDRAW');    time.sleep(3)
    c="SLEW "+ CRTrade;    print(c);    cdccmd(str.encode(c))
    time.sleep(3)
    cdccmd(b'REDRAW')
    time.sleep(3)
